# 1. Import and Install Dependencies

In [ ]:
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib

In [ ]:
pip install opencv-python mediapipe sklearn matplotlib

In [4]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# 2. Keypoints using MP Holistic

In [5]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilites

In [6]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)                # Make prediction
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [7]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
#     mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections    
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections    
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections    

In [11]:
cap = cv2.VideoCapture(0)
# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmakrs
        draw_landmarks(image, results)
        
        #Show to screen
        cv2.imshow('OpenCV Feed', image)
        
        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [ ]:
# len(results.left_hand_landmarks.landmark)

In [12]:
results

mediapipe.python.solution_base.SolutionOutputs

In [13]:
draw_landmarks(frame, results)

In [78]:
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

# 3. Extract Keypoint Values

In [ ]:
len(results.pose_landmarks.landmark)

In [ ]:
pose = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

In [ ]:
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

In [8]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [17]:
extract_keypoints(results).shape

(1662,)

In [18]:
468*3+33*4+21*3+21*3

1662

# 4. Setup Folders for Collection

In [9]:
# Path for exproted data, numpy arrays
DATA_PATH =os.path.join('MP_DATA')
# Actions that we try to detect
# actions = np.array(['hello', 'thanks', 'iloveu'])

actions = np.array(['Hongkong','Poland','Turkey','China','France','Brazil','Germany','Thai','US','Mexico'])
# Thirty videos worth of data
no_sequences = 10
# Videos are going to be 30 frames in length
sequence_length = 30

In [ ]:
# hello
## 0
## 1
## 2
## ...
## 29
# thanks

# I love you

In [ ]:
for action in actions:
    for sequence in range(no_sequences):
        try: os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# 5. Collect Keypoint Values for Training and Testing

In [ ]:
cap = cv2.VideoCapture(0)
# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video lenght aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)
                print(results)

                # Draw landmakrs
                draw_landmarks(image, results)

                # NEW Apply wait logic
                if frame_num == 0:
                    cv2.putText(image, 'STARTING COLLECTION', (120, 200),
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collection frames for {} Video Number {}'.format(action, sequence), (15, 12),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else:
                     cv2.putText(image, 'Collection frames for {} Video Number {}'.format(action, sequence), (15, 12),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)
                     # Show to screen
                     cv2.imshow('OpenCV Feed', image)

                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
cap.release()
cv2.destroyAllWindows()

# 6. Preprocess Data and Create Labels and Features

In [43]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [44]:
label_map = {label:num for num, label in enumerate(actions)}

In [45]:
label_map

{'Hongkong': 0,
 'Poland': 1,
 'Turkey': 2,
 'China': 3,
 'France': 4,
 'Brazil': 5,
 'Germany': 6,
 'Thai': 7,
 'US': 8,
 'Mexico': 9}

In [46]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [47]:
np.array(sequences).shape

(100, 30, 1662)

In [48]:
np.array(labels).shape

(100,)

In [49]:
X=np.array(sequences)
y=to_categorical(labels).astype(int)

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

# 7. Build and Train LSTM Neural Network

In [79]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [80]:
log_dir = os.path.join('Logs')
tb_callbacks = TensorBoard(log_dir=log_dir)

In [81]:
model= Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30, 1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [82]:
X.shape

(100, 30, 1662)

In [83]:
res = [0.7, 0.2, 0.1]

In [84]:
actions[np.argmax(res)]

'Hongkong'

In [85]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [88]:
# epochs edit avilable
model.fit(X_train, y_train, epochs=500, callbacks=[tb_callbacks])

Epoch 1/500
3/3 [==============================] - 0s 182ms/step - loss: 1.6421 - categorical_accuracy: 0.3684
Epoch 2/500
3/3 [==============================] - 0s 71ms/step - loss: 1.5907 - categorical_accuracy: 0.4211
Epoch 3/500
3/3 [==============================] - 0s 64ms/step - loss: 1.5976 - categorical_accuracy: 0.3684
Epoch 4/500
3/3 [==============================] - 0s 68ms/step - loss: 1.5807 - categorical_accuracy: 0.4737
Epoch 5/500
3/3 [==============================] - 0s 69ms/step - loss: 1.4931 - categorical_accuracy: 0.4211
Epoch 6/500
3/3 [==============================] - 0s 69ms/step - loss: 1.5859 - categorical_accuracy: 0.3684
Epoch 7/500
3/3 [==============================] - 0s 66ms/step - loss: 1.4462 - categorical_accuracy: 0.4526
Epoch 8/500
3/3 [==============================] - 0s 64ms/step - loss: 1.4502 - categorical_accuracy: 0.4632
Epoch 9/500
3/3 [==============================] - 0s 64ms/step - loss: 1.4824 - categorical_accuracy: 0.4316
Epoch 10/

3/3 [==============================] - 0s 73ms/step - loss: 1.0999 - categorical_accuracy: 0.5684
Epoch 75/500
3/3 [==============================] - 0s 71ms/step - loss: 1.1767 - categorical_accuracy: 0.5895
Epoch 76/500
3/3 [==============================] - 0s 67ms/step - loss: 1.1919 - categorical_accuracy: 0.4632
Epoch 77/500
3/3 [==============================] - 0s 65ms/step - loss: 1.0791 - categorical_accuracy: 0.5684
Epoch 78/500
3/3 [==============================] - 0s 68ms/step - loss: 1.0065 - categorical_accuracy: 0.6211
Epoch 79/500
3/3 [==============================] - 0s 67ms/step - loss: 1.0667 - categorical_accuracy: 0.5579
Epoch 80/500
3/3 [==============================] - 0s 66ms/step - loss: 1.1299 - categorical_accuracy: 0.5368
Epoch 81/500
3/3 [==============================] - 0s 69ms/step - loss: 1.0811 - categorical_accuracy: 0.5263
Epoch 82/500
3/3 [==============================] - 0s 64ms/step - loss: 1.0590 - categorical_accuracy: 0.6105
Epoch 83/500
3

3/3 [==============================] - 0s 69ms/step - loss: 0.9023 - categorical_accuracy: 0.6632
Epoch 148/500
3/3 [==============================] - 0s 67ms/step - loss: 0.9854 - categorical_accuracy: 0.6211
Epoch 149/500
3/3 [==============================] - 0s 69ms/step - loss: 0.8681 - categorical_accuracy: 0.6526
Epoch 150/500
3/3 [==============================] - 0s 67ms/step - loss: 0.8362 - categorical_accuracy: 0.6632
Epoch 151/500
3/3 [==============================] - 0s 69ms/step - loss: 0.8914 - categorical_accuracy: 0.6526
Epoch 152/500
3/3 [==============================] - 0s 76ms/step - loss: 0.9585 - categorical_accuracy: 0.6105
Epoch 153/500
3/3 [==============================] - 0s 64ms/step - loss: 0.9149 - categorical_accuracy: 0.6421
Epoch 154/500
3/3 [==============================] - 0s 63ms/step - loss: 0.8410 - categorical_accuracy: 0.6526
Epoch 155/500
3/3 [==============================] - 0s 66ms/step - loss: 0.9262 - categorical_accuracy: 0.5789
Epoch 

3/3 [==============================] - 0s 82ms/step - loss: 0.5528 - categorical_accuracy: 0.7684
Epoch 220/500
3/3 [==============================] - 0s 86ms/step - loss: 0.5933 - categorical_accuracy: 0.7684
Epoch 221/500
3/3 [==============================] - 0s 79ms/step - loss: 0.5082 - categorical_accuracy: 0.7789
Epoch 222/500
3/3 [==============================] - 0s 86ms/step - loss: 0.4963 - categorical_accuracy: 0.8211
Epoch 223/500
3/3 [==============================] - 0s 77ms/step - loss: 0.4673 - categorical_accuracy: 0.8737
Epoch 224/500
3/3 [==============================] - 0s 83ms/step - loss: 0.4626 - categorical_accuracy: 0.8737
Epoch 225/500
3/3 [==============================] - 0s 86ms/step - loss: 0.4666 - categorical_accuracy: 0.8105
Epoch 226/500
3/3 [==============================] - 0s 79ms/step - loss: 0.4430 - categorical_accuracy: 0.8737
Epoch 227/500
3/3 [==============================] - 0s 83ms/step - loss: 0.4525 - categorical_accuracy: 0.8842
Epoch 

3/3 [==============================] - 0s 85ms/step - loss: 0.5780 - categorical_accuracy: 0.8421
Epoch 293/500
3/3 [==============================] - 0s 81ms/step - loss: 0.6920 - categorical_accuracy: 0.7579
Epoch 294/500
3/3 [==============================] - 0s 86ms/step - loss: 0.7104 - categorical_accuracy: 0.7368
Epoch 295/500
3/3 [==============================] - 0s 79ms/step - loss: 0.6783 - categorical_accuracy: 0.8000
Epoch 296/500
3/3 [==============================] - 0s 87ms/step - loss: 0.7214 - categorical_accuracy: 0.7263
Epoch 297/500
3/3 [==============================] - 0s 86ms/step - loss: 0.5988 - categorical_accuracy: 0.7895
Epoch 298/500
3/3 [==============================] - 0s 86ms/step - loss: 0.6521 - categorical_accuracy: 0.6947
Epoch 299/500
3/3 [==============================] - 0s 89ms/step - loss: 0.5889 - categorical_accuracy: 0.7789
Epoch 300/500
3/3 [==============================] - 0s 90ms/step - loss: 0.5681 - categorical_accuracy: 0.7895
Epoch 

3/3 [==============================] - 0s 91ms/step - loss: 0.1835 - categorical_accuracy: 0.9474
Epoch 366/500
3/3 [==============================] - 0s 85ms/step - loss: 0.1818 - categorical_accuracy: 0.9474
Epoch 367/500
3/3 [==============================] - 0s 78ms/step - loss: 0.1763 - categorical_accuracy: 0.9474
Epoch 368/500
3/3 [==============================] - 0s 79ms/step - loss: 0.1644 - categorical_accuracy: 0.9474
Epoch 369/500
3/3 [==============================] - 0s 84ms/step - loss: 0.1765 - categorical_accuracy: 0.9368
Epoch 370/500
3/3 [==============================] - 0s 84ms/step - loss: 0.1794 - categorical_accuracy: 0.9474
Epoch 371/500
3/3 [==============================] - 0s 87ms/step - loss: 0.1714 - categorical_accuracy: 0.9263
Epoch 372/500
3/3 [==============================] - 0s 83ms/step - loss: 0.1669 - categorical_accuracy: 0.9474
Epoch 373/500
3/3 [==============================] - 0s 84ms/step - loss: 0.1647 - categorical_accuracy: 0.9368
Epoch 

3/3 [==============================] - 0s 91ms/step - loss: 2.1792 - categorical_accuracy: 0.5579
Epoch 438/500
3/3 [==============================] - 0s 96ms/step - loss: 2.7729 - categorical_accuracy: 0.5158
Epoch 439/500
3/3 [==============================] - 0s 91ms/step - loss: 1.7213 - categorical_accuracy: 0.5789
Epoch 440/500
3/3 [==============================] - 0s 92ms/step - loss: 1.2083 - categorical_accuracy: 0.6421
Epoch 441/500
3/3 [==============================] - 0s 91ms/step - loss: 1.2855 - categorical_accuracy: 0.6632
Epoch 442/500
3/3 [==============================] - 0s 97ms/step - loss: 0.8882 - categorical_accuracy: 0.7053
Epoch 443/500
3/3 [==============================] - 0s 90ms/step - loss: 0.7310 - categorical_accuracy: 0.7474
Epoch 444/500
3/3 [==============================] - 0s 92ms/step - loss: 0.5618 - categorical_accuracy: 0.8000
Epoch 445/500
3/3 [==============================] - 0s 80ms/step - loss: 1.0119 - categorical_accuracy: 0.6526
Epoch 

In [58]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 30, 64)            442112    
_________________________________________________________________
lstm_4 (LSTM)                (None, 30, 128)           98816     
_________________________________________________________________
lstm_5 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_3 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_4 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_5 (Dense)              (None, 10)                330       
Total params: 596,906
Trainable params: 596,906
Non-trainable params: 0
________________________________________________

# 8. MakePredictions

In [59]:
res=model.predict(X_test)

In [49]:
actions[np.argmax(res[0])]

'hello'

In [50]:
actions[np.argmax(y_test[4])]

IndexError: index 4 is out of bounds for axis 0 with size 2

# 9. Save Model Weights

In [60]:
model.save('action.h5')

In [52]:
# del model

In [61]:
model.load_weights('action.h5')

# 10. Evaluation using a Confusing Matrix

In [62]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [63]:
yhat = model.predict(X_train)

In [50]:
yhat

array([[5.30937910e-01, 2.04264224e-01, 2.64797777e-01],
       [8.46328020e-01, 1.33085594e-01, 2.05863640e-02],
       [6.64181784e-02, 1.37740493e-01, 7.95841277e-01],
       [1.16920114e-01, 4.72715676e-01, 4.10364211e-01],
       [7.11169541e-01, 2.11421192e-01, 7.74093047e-02],
       [2.67028421e-01, 7.25050449e-01, 7.92112295e-03],
       [4.41977024e-01, 4.84891206e-01, 7.31317028e-02],
       [2.90677607e-01, 6.29515409e-01, 7.98070729e-02],
       [1.65860742e-01, 2.16442049e-01, 6.17697239e-01],
       [1.18185185e-01, 8.78730536e-01, 3.08427797e-03],
       [1.69843420e-01, 6.75639749e-01, 1.54516831e-01],
       [7.94932172e-02, 9.84264985e-02, 8.22080314e-01],
       [1.59950078e-01, 8.35830748e-01, 4.21916973e-03],
       [2.15876535e-01, 1.23923503e-01, 6.60199940e-01],
       [1.89573243e-02, 2.62561738e-01, 7.18480885e-01],
       [9.44725499e-02, 1.80708304e-01, 7.24819183e-01],
       [4.64029700e-01, 2.44944736e-01, 2.91025549e-01],
       [2.41452202e-01, 7.49523

In [64]:
ytrue = np.argmax(y_train, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [65]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[86,  0],
        [ 9,  0]],

       [[85,  0],
        [10,  0]],

       [[85,  0],
        [10,  0]],

       [[85,  0],
        [10,  0]],

       [[85,  0],
        [10,  0]],

       [[86,  0],
        [ 9,  0]],

       [[85,  0],
        [10,  0]],

       [[ 0, 86],
        [ 0,  9]],

       [[86,  0],
        [ 9,  0]],

       [[86,  0],
        [ 9,  0]]], dtype=int64)

In [66]:
accuracy_score(ytrue, yhat)

0.09473684210526316

# 11. Test in Real Time

In [67]:
from scipy import stats

In [72]:
colors = [(16,117,245), (117,245,16), (16,117,245),(117,245,16), (16,117,245), 
         (117,245,16), (16,117,245), (117,245,16), (16,117,245), (117,245,16)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
#         print(num)
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [ ]:
# plt.figure(figsize=(18,18))
# plt.show(prob_viz(res, actions, image, colors))

In [77]:
# 1. New detection variables
sequence =[]
sentence =[]
predictions = []
threshold =0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        
        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
#         print(results)

        # Draw landmakrs
        draw_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0, keypoints)
        sequence.append(keypoints)
        sequence=sequence[-30:]
        
        if len(sequence)==30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))

            # 3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res):
                if res[np.argmax(res)] > threshold:
                    
                    if len(sentence) >0:
                         if actions[np.argmax(res)] != sentence[-1]:
                                sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5:
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions ,image, colors)
        
        cv2.rectangle(image, (0,0), (640,40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    
        # Show to screen
        cv2.imshow('OpenCV Feed', image)
        
        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
                
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thai
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thai
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thai
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thai
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thai
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thai
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thai
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thai
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thai
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thai
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thai
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thai
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thai
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thai
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thai
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thai
<class '

In [73]:
cap.release()
cv2.destroyAllWindows()

In [74]:
extract_keypoints(results)

array([ 0.60914385,  0.53727126, -0.96616203, ...,  0.        ,
        0.        ,  0.        ])

In [75]:
sequence.insert(0, keypoints)
sequence

[array([ 0.60914385,  0.53727126, -0.96616203, ...,  0.        ,
         0.        ,  0.        ]),
 array([ 0.59188932,  0.58113408, -0.85732049, ...,  0.        ,
         0.        ,  0.        ]),
 array([ 0.59189582,  0.58113134, -0.92448014, ...,  0.        ,
         0.        ,  0.        ]),
 array([ 0.59187752,  0.58117688, -0.82181406, ...,  0.        ,
         0.        ,  0.        ]),
 array([ 0.59205049,  0.58109117, -0.73144394, ...,  0.        ,
         0.        ,  0.        ]),
 array([ 0.59184331,  0.58105111, -0.80679005, ...,  0.        ,
         0.        ,  0.        ]),
 array([ 0.58876586,  0.58151233, -0.7504729 , ...,  0.        ,
         0.        ,  0.        ]),
 array([ 0.58869499,  0.57998002, -0.89222968, ...,  0.        ,
         0.        ,  0.        ]),
 array([ 0.59040701,  0.57369727, -0.87971288, ...,  0.        ,
         0.        ,  0.        ]),
 array([ 0.59190887,  0.5644204 , -0.86720961, ...,  0.        ,
         0.        ,  0.  